In [1]:
import pandas as pd
import numpy as np 

In [2]:
df = pd.read_excel("recomendation.xls")
df

,Unnamed: 0,TRUE BELIEVER,THE DA VINCI CODE,THE WORLD IS FLAT,MY LIFE SO FAR,THE TAKING,THE KITE RUNNER,RUNNY BABBIT,HARRY POTTER
0,U1,2,4,,3,NaN,,1.0,1.0
1,U2,NaN,5,,,3.0,2,1.0,NaN
2,U3,3,,1,2,2.0,NaN,NaN,5.0
3,U4,NaN,3,NaN,NaN,4.0,1,NaN,3.0
4,U5,2,4,3,NaN,NaN,2,1.0,NaN
5,U6,5,4,NaN,3,1.0,NaN,3.0,1.0
6,U7,1,4,5,5,3.0,NaN,NaN,4.0
7,U8,2,2,NaN,NaN,4.0,5,1.0,NaN
8,U9,,NaN,4,2,1.0,NaN,NaN,5.0
9,U10,3,5,1,NaN,NaN,NaN,4.0,4.0


In [3]:
# preprocessing
df = df.replace({" ":np.nan}) ## replace " " with nan  
df = df.to_numpy() # convert from dataframe to np array
df = df[:,1:].astype(float) # convert variable type to float
mean = np.nanmean(df, axis=1,keepdims=True) # calculate mean ignoring nans
df

C:\Users\ALIGUZ~1\AppData\Local\Temp/ipykernel_1284/629323724.py:5: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(df, axis=1,keepdims=True) # calculate mean ignoring nans


array([[ 2.,  4., nan,  3., nan, nan,  1.,  1.],
       [nan,  5., nan, nan,  3.,  2.,  1., nan],
       [ 3., nan,  1.,  2.,  2., nan, nan,  5.],
       [nan,  3., nan, nan,  4.,  1., nan,  3.],
       [ 2.,  4.,  3., nan, nan,  2.,  1., nan],
       [ 5.,  4., nan,  3.,  1., nan,  3.,  1.],
       [ 1.,  4.,  5.,  5.,  3., nan, nan,  4.],
       [ 2.,  2., nan, nan,  4.,  5.,  1., nan],
       [nan, nan,  4.,  2.,  1., nan, nan,  5.],
       [ 3.,  5.,  1., nan, nan, nan,  4.,  4.],
       [ 2., nan,  2.,  4., nan,  1., nan,  2.],
       [ 5.,  4., nan,  2., nan,  1.,  1.,  3.],
       [nan, nan,  2., nan,  4., nan,  4.,  5.],
       [nan,  1.,  2.,  3.,  4., nan,  5.,  5.],
       [nan,  3., nan, nan,  5.,  3., nan,  2.],
       [nan,  3.,  2.,  1.,  1., nan,  4., nan],
       [ 1.,  5.,  1.,  2., nan,  4., nan,  4.],
       [ 5., nan,  4., nan,  2.,  1.,  3.,  5.],
       [nan,  3., nan,  2., nan,  4.,  1.,  4.],
       [ 2.,  5.,  1.,  1.,  5.,  4., nan,  4.],
       [nan, nan, na

In [4]:
# calculate sim(a,b) = cov(a,b)/(sd(a)*sd(b)) only if a and b is not nan 
sim =np.zeros((df.shape[0],df.shape[0]))

for i in range(df.shape[0]):
    for j in range(df.shape[0]):
        if i==j:
            sim[i,j]= 0
        else:
            cov = 0
            stda = 0
            stdb = 0
            for k in range(df.shape[1]):
                if  ~np.isnan(df[i,k]) and ~np.isnan(df[j,k]):
                    cov += (df[i,k]-mean[i])*(df[j,k]-mean[j])
                    stda += (df[i,k]-mean[i])**2
                    stdb += (df[j,k]-mean[j])**2
            sim[i,j] = cov/(np.sqrt(stda)*np.sqrt(stdb))   

C:\Users\ALIGUZ~1\AppData\Local\Temp/ipykernel_1284/2716135937.py:17: RuntimeWarning: invalid value encountered in double_scalars
  sim[i,j] = cov/(np.sqrt(stda)*np.sqrt(stdb))


In [5]:
def getMostKSimilarUsers(user_index, k, sim):
    tmp = sim[user_index]
    tmp = tmp[0:20]   
    return  (-tmp).argsort()[:k]

In [6]:
def getBooksPrediction(user_index,most_similar_users):
    pred=[]
    for i, b in enumerate(df[user_index]):
        if np.isnan(b):
            print("Unread book:{}".format(i))
            numerator = 0
            sum_of_sim = 0
            for j,msu in enumerate(most_similar_users):
                if ~np.isnan(df[msu,i]):
                    numerator += sim[user_index,msu]*(df[msu,i]-mean[msu])
                    sum_of_sim += sim[user_index,msu]
            if sum_of_sim!=0:
                p = mean[user_index]+(numerator/sum_of_sim)
                pred.append((i,p[0]))
    return pred

In [7]:
def suggestBook(pred):
    books =["TRUE BELIEVER","THE DA VINCI CODE","THE WORLD IS FLAT","MY LIFE SO FAR","THE TAKING","THE KITE RUNNER","RUNNY BABBIT","HARRY POTTER"]
    best_rate = np.NINF
    suggested_book  = np.NINF
    
    if len(pred)==0:
        print("No suggested book")
        return
    
    for i in range(len(pred)):
        if (pred[i][1]>best_rate):
            best_rate = pred[i][1]
            suggested_book = pred[i][0]
    print("Suggessted book: {}".format(books[suggested_book]))
    return books[suggested_book]

In [8]:
def displayMostSimilarUsers(user_index, msu):
    users = ["U1","U2","U3","U4","U5","U6","U7","U8","U9","U10","U11","U12","U13","U14","U15","U16","U17","U18","U19","U20"]
    for i,m in enumerate(msu):
        print("Most Similar Users:{} sim ratio:{}".format(users[m], sim[user_index,m]))

In [9]:
# advice book for k=1,3
for i in range(21,26):
    for j in range(1,4,2):
        print("users:{} k:{}".format(i,j))
        user_index = i
        k= j
        most_similar_users = getMostKSimilarUsers(user_index,k,sim)
        displayMostSimilarUsers(user_index, most_similar_users)

        pred= getBooksPrediction(user_index,most_similar_users)
        print("pred: {}".format(pred))
        suggestBook(pred)
        print("\n")

users:21 k:1
Most Similar Users:U9 sim ratio:0.8485281374238569
Unread book:1
Unread book:6
pred: []
No suggested book


users:21 k:3
Most Similar Users:U9 sim ratio:0.8485281374238569
Most Similar Users:U12 sim ratio:0.8438159256945179
Most Similar Users:U18 sim ratio:0.7001400420140048
Unread book:1
Unread book:6
pred: [(1, 4.833333333333334), (6, 2.437962011574829)]
Suggessted book: THE DA VINCI CODE


users:22 k:1
Most Similar Users:U2 sim ratio:0.9612102264841017
Unread book:0
Unread book:5
Unread book:7
pred: [(5, 2.65)]
Suggessted book: THE KITE RUNNER


users:22 k:3
Most Similar Users:U2 sim ratio:0.9612102264841017
Most Similar Users:U1 sim ratio:0.9515692560743098
Most Similar Users:U11 sim ratio:0.8206934047378385
Unread book:0
Unread book:5
Unread book:7
pred: [(0, 3.1999999999999997), (5, 2.44274298247051), (7, 2.6630766211379475)]
Suggessted book: TRUE BELIEVER


users:23 k:1
Most Similar Users:U16 sim ratio:0.5105835863457981
Unread book:2
Unread book:3
pred: [(2, 1.9666

In [10]:
## save sim() table
sim= np.around(sim,decimals=3)
df_sim = pd.DataFrame(sim)
print("{}".format(df_sim))
df_sim.to_excel("output.xlsx") 

       0      1      2      3      4      5      6      7      8      9   ...  \
0   0.000  0.997 -0.943  0.196  0.987  0.473  0.258  0.191 -0.992  0.330  ...   
1   0.997  0.000 -1.000  0.424  0.996  0.301  0.346  0.000 -1.000  0.524  ...   
2  -0.943 -1.000  0.000 -0.048 -0.942 -0.291 -0.276 -1.000  0.526  0.702  ...   
3   0.196  0.424 -0.048  0.000  0.377 -0.666 -0.629 -0.448 -0.555  0.910  ...   
4   0.987  0.996 -0.942  0.377  0.000  0.144  0.456  0.101  1.000  0.066  ...   
5   0.473  0.301 -0.291 -0.666  0.144  0.000 -0.412 -0.689 -0.021 -0.000  ...   
6   0.258  0.346 -0.276 -0.629  0.456 -0.412  0.000  0.234  0.312 -0.156  ...   
7   0.191  0.000 -1.000 -0.448  0.101 -0.689  0.234  0.000 -1.000 -0.547  ...   
8  -0.992 -1.000  0.526 -0.555  1.000 -0.021  0.312 -1.000  0.000 -0.217  ...   
9   0.330  0.524  0.702  0.910  0.066 -0.000 -0.156 -0.547 -0.217  0.000  ...   
10  0.648  1.000 -0.242  0.953  0.433  0.045  0.432 -0.871 -0.538  0.507  ...   
11  0.354  0.898  0.348  0.8